In [12]:
import cv2
from matplotlib import pyplot as plt

In [13]:
config_file = "ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"
frozen_graph = "frozen_inference_graph.pb"

In [14]:
model = cv2.dnn_DetectionModel(frozen_graph, config_file)

In [15]:
classLabels = []  
filename = 'coco_labels.txt'
with open(filename, 'rt') as spt:
    classLabels = spt.read().rstrip('\n').split('\n')
    #classlabels.append(fpt.append)

In [16]:
print(classLabels)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [17]:
print(len(classLabels))

80


In [18]:
model.setInputSize(320, 320) #greater this value better the reults tune it for best output
model.setInputScale(1.0/127.5)
model.setInputMean((127.5, 127.5, 127.5))
model.setInputSwapRB(True) # --> this automatically changes the image from BGR to RGB

< cv2.dnn.Model 0000020BFF5FB670>

### video

In [19]:
# Initialize the video capture object
cap = cv2.VideoCapture('KRL_43sec.mp4')  # Input video file

# Check if the video file is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open video")

# Desired display size
display_width = 1200  # Adjust the width of the display window
display_height = 600  # Adjust the height of the display window

font = cv2.FONT_HERSHEY_PLAIN

# Threshold for object size (example values, adjust as needed)
MIN_CAR_SIZE = 500  # Minimum area for cars/trucks
MAX_CAR_SIZE = 20000  # Maximum area for cars/trucks

def refine_class(label, box):
    """Refine class based on bounding box size."""
    box_area = box[2] * box[3]  # width * height
    if label == "train":
        if MIN_CAR_SIZE <= box_area <= MAX_CAR_SIZE:
            return "vehicle"
    return label
    
try:
    while True:
        ret, frame = cap.read()  # Read a frame from the video

        # Break the loop if no frame is read (end of video)
        if not ret:
            print("End of video reached or cannot fetch the frame.")
            break

        # Detect objects (example detection function)
        classIndex, confidence, bbox = model.detect(frame, confThreshold=0.46)  # Tune the confidence as required

        # Draw bounding boxes and labels if detections exist
        if len(classIndex) != 0:
            for classInd, boxes in zip(classIndex.flatten(), bbox):
                label = classLabels[classInd - 1]
                label = refine_class(label, boxes)  # Refine classification
                
                cv2.rectangle(frame, boxes, (255, 0, 0), 2)
                cv2.putText(frame, label, 
                            (boxes[0] + 10, boxes[1] + 40), 
                            font, fontScale=1, color=(0, 255, 0), thickness=2)
                
        # Resize the frame for display
        resized_frame = cv2.resize(frame, (display_width, display_height))

        # Display the frame with detections
        cv2.imshow('Object Detection Tutorial', resized_frame)

        # Check for the 'q' key to break the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Terminating program...")
            break

finally:
    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    print("Resources released successfully.")

Terminating program...
Resources released successfully.
